In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
import cv2

from tensorflow.keras import layers
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from sklearn.model_selection import train_test_split

In [2]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

In [3]:
train_dir = "./asl"

In [4]:
data_generator = ImageDataGenerator(
    samplewise_center=True, 
    samplewise_std_normalization=True,
    brightness_range=[0.8, 1.0],
    zoom_range=[1.0, 1.2],
    validation_split=0.2
)

train_generator = data_generator.flow_from_directory(train_dir, target_size=(200, 200), shuffle=True, seed=13,
                                                     class_mode='categorical', batch_size=32, subset="training")

validation_generator = data_generator.flow_from_directory(train_dir, target_size=(200, 200), shuffle=True, seed=13,
                                                          class_mode='categorical', batch_size=32, subset="validation")

KeyboardInterrupt: 

In [ ]:
WEIGHTS_FILE = './inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

inception_v3_model = keras.applications.inception_v3.InceptionV3(
    input_shape = (200, 200, 3), 
    include_top = False, 
    weights = 'imagenet'
)

In [ ]:
inception_output_layer = inception_v3_model.get_layer('mixed7')
print('Inception model output shape:', inception_output_layer.output_shape)

inception_output = inception_v3_model.output

In [ ]:
x = layers.GlobalAveragePooling2D()(inception_output)
x = layers.Dense(1024, activation='relu')(x)                  
x = layers.Dense(17, activation='softmax')(x)           

model = Model(inception_v3_model.input, x) 

model.compile(
    optimizer=SGD(lr=0.0001, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

In [ ]:
LOSS_THRESHOLD = 0.2
ACCURACY_THRESHOLD = 0.95

class ModelCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('val_loss') <= LOSS_THRESHOLD and logs.get('val_acc') >= ACCURACY_THRESHOLD:
            print("\nReached", ACCURACY_THRESHOLD * 100, "accuracy, Stopping!")
            self.model.stop_training = True

callback = ModelCallback()

In [ ]:
history = model.fit(
            train_generator,
            validation_data=validation_generator,
            steps_per_epoch=200,
            validation_steps=50,
            callbacks=[callback],
            epochs=50
          )

In [ ]:
print("-- Evaluate --")

scores = model.evaluate_generator(validation_generator, steps=5)

model.save('asl-model-2.h5')

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

In [ ]:
print("-- Predict --")

img = image.load_img('./asl/Baby/Baby_100.jpg')

plt.imshow(img)

img = (np.expand_dims(img, 0))

output = model.predict(img)

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

print(validation_generator.class_indices)

print(output)